# Stitching back together - function

In [6]:
# We have an input of a numpy array with one dimension being: 'bounding box'
# shape (num_rows, 4)

In [25]:
import numpy as np
import pandas as pd

In [8]:
ex_bbs = np.random.randint(70, 640, (10, 4))
ex_bbs

array([[  0, 270,  52, 316],
       [ 68, 155, 523, 368],
       [541, 343, 109, 243],
       [244, 320, 166,  86],
       [333, 369, 372, 168],
       [188, 242, 231, 124],
       [488, 508, 183, 505],
       [153,  15, 150, 416],
       [195, 366,  98,  45],
       [485, 453, 295, 202]])

# Function: **luc_coordinates**

In [9]:
def luc_coordinates(num_width_comp, num_height_comp, comp_size):
  ''' A function that takes as an input:
  width_comp = number of compartments for the width (e.g. 8)
  height_comp = number of compartments for the height (e.g. 6)
  comp_size = size of each compartment (e.g. 640)
  and returns the coordinates of the left-upper-corner (luc) of
  each compartment
  '''

  # Height and width of the input image
  width = 4000
  height = 3000

  # Step size is the size of the compartments unique pixels horizontally and vertically
  step_size_horiz = width / num_width_comp
  step_size_vert = height / num_height_comp

  # Defining how much overlap there will be between each compartment vertically and horizontally
  overlap_horiz = int(((comp_size * num_width_comp) - width)/num_width_comp)
  overlap_vert = int(((comp_size * num_height_comp) - height)/num_height_comp)

  #creating a list with all the x_coordinates for the compartments (8)
  x_coordinates = [0]
  [x_coordinates.append(int(ele * step_size_horiz)) for ele in range(1, num_width_comp)]

  # creating a list with all the y_coordinates for the compartments (6)
  y_coordinates = [0]
  [y_coordinates.append(int(ele * step_size_vert)) for ele in range(1, num_height_comp)]


  # Creating a dictionary with the coordinates
  coordinates_dict = {}
  b = 0 # no inherent important meaning

  for ele in range (0, num_height_comp):
    for i in range (0, num_width_comp):
      a = [x_coordinates[i], y_coordinates[ele]]
      b += 1
      coordinates_dict[f'cor_{b}'] = a

  return coordinates_dict

# Function: **slicing_dictionary**

In [16]:
def slicing_dictionary(tensor):

  # Calling the function 'luc_coordinates' and saving the resulting dictionary of coordinates in a variable
  # coordinates_dict = luc_coordinates(num_width_comp, num_height_comp, comp_size)

  slize_size = 640

  # Turning the coordinates into a list
  coordinates_list = [value for key, value in coordinates_dict.items()]

  # creating a for loop for getting the correct slizing integers and putting them into a dictionary
  slicing_dict = {}

  for i in range (0, 48):
    slice_1, slice_2 = coordinates_list[i][0], coordinates_list[i][1]
    slice_1_a = slice_1
    slice_1_b = slice_1_a + slize_size
    slice_2_a = slice_2
    slice_2_b = slice_2_a + slize_size
    slicing_dict[i] = [slice_1_a, slice_1_b, slice_2_a, slice_2_b]

  return slicing_dict

# Function: pipeline_input_to_slicing_dict

In [14]:
# Pipeline from Input-tensor to slicing_dict + cp_bb

def pipeline_input_to_slicing_dict():

  # Calling the function 'luc_coordinates'
  coordinates_dict = luc_coordinates(num_width_comp, num_height_comp, comp_size)

  #Calling the function 'slicing_dictionary'
  slicing_dict = slicing_dictionary(tensor)

  # Image-level: Getting the bounding box coordinates ONCE for the whole image
  corner_points_of_bounding_boxes_original_image_with_padding = {}

  for i in range(0, df.shape[0]):

    bb = i
    #function: getting_bounding_boxes
    x_min, y_min, x_max, y_max, class_object = getting_bounding_boxes(width_img, height_img, comp_size, bb)
    corner_points_of_bounding_boxes_original_image_with_padding[bb] = [x_min, y_min, x_max, y_max, class_object]

  cp_bb = corner_points_of_bounding_boxes_original_image_with_padding  # = dict
  cp_bb

  return slicing_dict, cp_bb


# Pipeline (1)

In [11]:
# Needed Variables

#for 'luc_coordinates'
num_width_comp = 8
num_height_comp = 6
comp_size = 640

#for 'slicing_dict'
tensor = np.ones((4140, 3140, 3))

#for 'getting_bounding_boxes'
width_img = 4000
height_img = 3000
#comp_size

In [ ]:
coordinates_dict = luc_coordinates(num_width_comp, num_height_comp, comp_size)

slicing_dict = slicing_dictionary(tensor)
# slicing_dict, cp_bb

coordinates_dict, slicing_dict

In [55]:
# Need a function that gets me 48 DataFrames out of a 'chunk' of 48 bounding
# box coordinates. Store them in a dictionary with those DataFrames
# -> then I can work with them

ex_bbs

examp_df = pd.DataFrame(ex_bbs)
examp_df.columns = ['x_min', 'y_min', 'x_max', 'y_max']
examp_df

,x_min,y_min,x_max,y_max
0,0,270,52,316
1,68,155,523,368
2,541,343,109,243
3,244,320,166,86
4,333,369,372,168
5,188,242,231,124
6,488,508,183,505
7,153,15,150,416
8,195,366,98,45
9,485,453,295,202


In [ ]:
# Create an empty DataFrame with the specified columns
columns = ['class', 'x_min', 'y_min', 'x_max', 'y_max']
data_frame = pd.DataFrame(columns=columns)
data_frame

# Running this once, so that I have a dictionary full of names for the dataframes that come out of my loops
dict_df_comps = {}

for i in range (0, 48):
    dict_df_comps[i] = examp_df

dict_df_comps

# Re-stitching-functions

In [58]:
# example: key = 17

def re_to_absolute_coordinates_xyxy(dict_of_dfs):
    '''
    Input: A dictionary with 48 Dataframes as Value and range (0, 48) as keys
    slicing_dict has to be defined as well
    Calculates the absolute coordinates of a bounding box
    for a single compartment in the context of the output
    image of size width = 4000, height = 3000
    Output: absolute coordinates in form (xyxy = x_min, y_min, x_max, y_max)
    Those need to be blurred
    '''

    # Defining empty dataframe with the right column-names
    new_df = pd.DataFrame(columns=['x_min', 'y_min', 'x_max', 'y_max'])

    for key, value in dict_of_dfs.items():
        # key represents the number of the compartment
        # value represents the Dataframe with bounding box coordinates

        for num in range(0, value.shape[0]):

            abs_x_min = (value.iloc[num][0] + slicing_dict[key][0]) - 70
            abs_y_min = (value.iloc[num][1] + slicing_dict[key][2]) - 70
            abs_x_max = (value.iloc[num][2] + slicing_dict[key][0]) - 70
            abs_y_max = (value.iloc[num][3] + slicing_dict[key][2]) - 70

            new_df.loc[len(new_df)] = abs_x_min, abs_y_min, abs_x_max, abs_y_max

    return new_df       # Puts out one big Dataframe with all bounding boxes
                        # to be blurred

In [59]:
new_df = re_to_absolute_coordinates_xyxy(dict_of_dfs=dict_df_comps)

In [61]:
new_df

,x_min,y_min,x_max,y_max
0,-70,200,-18,246
1,-2,85,453,298
2,471,273,39,173
3,174,250,96,16
4,263,299,302,98
...,...,...,...,...
475,3618,2672,3661,2554
476,3918,2938,3613,2935
477,3583,2445,3580,2846
478,3625,2796,3528,2475
